<a href="https://colab.research.google.com/github/joacorapela/svGPFA/blob/master/doc/ipynb/doEstimateAndPlot_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contents:
- [1 Estimation](#estimation)
- [2 Plotting](#plotting)
- [3 GOF](#GOF)

# Clone repository

In [1]:
!rm -rf svGPFA/
!git clone -b master https://github.com/joacorapela/svGPFA.git
%pip install git+file:///content/svGPFA
%cd svGPFA/doc/ipynb

Cloning into 'svGPFA'...
remote: Enumerating objects: 9408, done.
remote: Counting objects: 100% (271/271), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 9408 (delta 136), reused 215 (delta 96), pack-reused 9137
Receiving objects: 100% (9408/9408), 256.26 MiB | 18.44 MiB/s, done.
Resolving deltas: 100% (5677/5677), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning file:///content/svGPFA to /tmp/pip-req-build-yusorki5
  Running command git clone -q file:///content/svGPFA /tmp/pip-req-build-yusorki5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for svGPFA: filename=svGPFA-1.0.0-py3-none-any.whl size=55511 sha256=66d9b318deb9e00fa12a14c21edc11376738eec18ee6e97e9173dee6b0b3b7cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-pdove8m1/wheels/fd/52/31/beff060e4ee2d

# 1 Estimation <a class="anchor" id="estimation"></a>

## 1.1 Import requirements

In [2]:
import time
import torch
import pickle
import plotly.colors

import svGPFA.stats.kernels
import svGPFA.stats.svGPFAModelFactory
import svGPFA.stats.svEM
import svGPFA.utils.miscUtils
import svGPFA.utils.initUtils
import gcnu_common.stats.pointProcesses.tests

## 1.2 Set parameters

In [3]:
n_latents = 2                                      # number of latents
n_neurons = 100                                    # number of neurons
n_trials = 15                                      # number of trials
prior_cov_reg_param = 1e-5                         # prior covariance regularization parameter
trials_start_time = 0.0                             # trial start time
trials_end_time = 1.0                               # trial end time
em_max_iter = 30                                   # maximum number of EM iterations

## 1.3 Load spikes times

In [4]:
simResFilename = "../../examples/data/32451751_simRes.pickle" # simulation results filename
with open(simResFilename, "rb") as f:
    simRes = pickle.load(f)
spikes_times = simRes["spikes"]

## 1.4 Create initial values of parameters

In [5]:
# refer to this https://joacorapela.github.io/svGPFA/params.html# for a full description
# of different ways of specify svGPFA parameters

# get default params
default_params = svGPFA.utils.initUtils.getDefaultParamsDict(
    n_neurons=n_neurons, n_trials=n_trials, n_latents=n_latents,
    trials_start_time=trials_start_time, trials_end_time=trials_end_time,
    em_max_iter=em_max_iter, prior_cov_reg_param=prior_cov_reg_param)

# build the svGPFA parameters from their default and dynamic specifications
initial_params, quad_params, kernels_types, optim_params = \
    svGPFA.utils.initUtils.getParams(
        n_trials=n_trials, n_neurons=n_neurons,
        default_params=default_params)
kernels_params0 = initial_params["svPosteriorOnLatents"]["kernelsMatricesStore"]["kernelsParams0"]
optim_method = optim_params["optim_method"]
prior_cov_reg_param = optim_params["prior_cov_reg_param"]

Extracted default_params[model_structure_params][n_latents]=2
Extracted default_params[optim_params][n_quad]=200
Extracted default_params[ind_points_params0][n_ind_points]=10
Extracted c0 from default
Extracted d0 from default
Extracted trials_start_times from default
Extracted trials_end_times from default
Extracted k_type=exponentialQuadratic and k_lengthsale0=1.0 from default
Extracted ind_points_locs0_layout=equidistant from default
Extracted variational_mean0 from default
Extracted variational_cov0 from default
Extracted default_params[optim_params][n_quad]=200
Extracted default_params[optim_params][prior_cov_reg_param]=1e-05
Extracted default_params[optim_params][optim_method]=ecm
Extracted default_params[optim_params][em_max_iter]=30
Extracted default_params[optim_params][verbose]=True
Extracted default_params[optim_params][estep_estimate]=True
Extracted default_params[optim_params][estep_max_iter]=20
Extracted default_params[optim_params][estep_lr]=1.0
Extracted default_params[

## 1.5 Create a model and set its initial parameters

In [6]:
kernels = svGPFA.utils.miscUtils.buildKernels(
    kernels_types=kernels_types, kernels_params=kernels_params0)

kernelMatrixInvMethod = svGPFA.stats.svGPFAModelFactory.kernelMatrixInvChol
indPointsCovRep = svGPFA.stats.svGPFAModelFactory.indPointsCovChol
model = svGPFA.stats.svGPFAModelFactory.SVGPFAModelFactory.buildModelPyTorch(
    conditionalDist=svGPFA.stats.svGPFAModelFactory.PointProcess,
    linkFunction=svGPFA.stats.svGPFAModelFactory.ExponentialLink,
    embeddingType=svGPFA.stats.svGPFAModelFactory.LinearEmbedding,
    kernels=kernels, kernelMatrixInvMethod=kernelMatrixInvMethod,
    indPointsCovRep=indPointsCovRep)
model.setInitialParamsAndData(
    measurements=spikes_times,
    initialParams=initial_params,
    eLLCalculationParams=quad_params,
    priorCovRegParam=prior_cov_reg_param)

## 1.7 Maximize the Lower Bound
<span style="color:red">(Warning: with the parameters above, this step takes around 10 minutes)</span>

In [ ]:
svEM = svGPFA.stats.svEM.SVEM_PyTorch()
tic = time.perf_counter()
lowerBoundHist, elapsedTimeHist, terminationInfo, iterationsModelParams = \
    svEM.maximize(model=model, optim_params=optim_params, method=optim_method)
toc = time.perf_counter()
print(f"Elapsed time {toc - tic:0.4f} seconds")

Iteration 01, estep start: -inf
Iteration 01, estep end: 126319.588307, niter: 19, nfeval: 26
Iteration 01, mstep_embedding start: 126319.588307
Iteration 01, mstep_embedding end: 906612.882092, niter: 13, nfeval: 25
Iteration 01, mstep_kernels start: 906612.882092


# 2 Plotting <a class="anchor" id="plotting"></a>

## 2.1 Setup for plotting

In [ ]:
import numpy as np
import pandas as pd
import sklearn.metrics
import svGPFA.plot.plotUtilsPlotly

trials_colors = plotly.colors.n_colors('rgb(0, 0, 255)', 'rgb(255, 0, 0)', n_trials, colortype = 'rgb')

## 2.2 Lower bound history

In [ ]:
fig = svGPFA.plot.plotUtilsPlotly.getPlotLowerBoundHist(lowerBoundHist=lowerBoundHist)
fig.show()

## 2.3 Set neuron, latent and times to plot

In [ ]:
neuronToPlot = 0
latentToPlot = 0
sampling_rate = 1000.0 # Hz
trials_times = torch.arange(trials_start_time, trials_end_time, 1.0/sampling_rate)

## 2.4 Latents

In [ ]:
# plot estimated latent across trials
testMuK, testVarK = model.predictLatents(times=trials_times)
indPointsLocs = model.getIndPointsLocs()
fig = svGPFA.plot.plotUtilsPlotly.getPlotLatentAcrossTrials(times=trials_times.numpy(), latentsMeans=testMuK, latentsSTDs=torch.sqrt(testVarK), indPointsLocs=indPointsLocs, latentToPlot=latentToPlot, trials_colors=trials_colors, xlabel="Time (msec)")
fig.show()

## 2.5 Embedding

In [ ]:
embeddingMeans, embeddingVars = model.predictEmbedding(times=trials_times)
embeddingMeans = embeddingMeans.detach().numpy()
embeddingVars = embeddingVars.detach().numpy()
title = "Neuron {:d}".format(neuronToPlot)
fig = svGPFA.plot.plotUtilsPlotly.getPlotEmbeddingAcrossTrials(times=trials_times.numpy(), embeddingsMeans=embeddingMeans[:,:,neuronToPlot], embeddingsSTDs=np.sqrt(embeddingVars[:,:,neuronToPlot]), trials_colors=trials_colors, title=title)
fig.show()

## 2.6 CIFs

In [ ]:

with torch.no_grad():
    ePosCIFValues = model.computeExpectedPosteriorCIFs(times=trials_times)
fig = svGPFA.plot.plotUtilsPlotly.getPlotCIFsOneNeuronAllTrials(times=trials_times, cif_values=ePosCIFValues, neuron_index=neuronToPlot, trials_colors=trials_colors)                                                                                                                                      
fig.show()

## 2.7 Embedding parameters

In [ ]:
estimatedC, estimatedD = model.getSVEmbeddingParams()
fig = svGPFA.plot.plotUtilsPlotly.getPlotEmbeddingParams(C=estimatedC.numpy(), d=estimatedD.numpy())
fig.show()

## 2.8 Kernels parameters

In [ ]:
kernelsParams = model.getKernelsParams()
kernelsTypes = [type(kernel).__name__ for kernel in model.getKernels()]
fig = svGPFA.plot.plotUtilsPlotly.getPlotKernelsParams(
    kernelsTypes=kernelsTypes, kernelsParams=kernelsParams)
fig.show()

# 3 Goodness of fit (GOF) tests <a class="anchor" id="GOF"></a>


## 3.1 Set trial and neuron for GOF assesment

In [ ]:
trialForGOF = 0
neuronForGOF = 0

## 3.2 KS time-rescaling GOF test

In [ ]:
ksTest_gamma = 20                                 # number of simulations for the KS test numerical correction
with torch.no_grad():
    epmcifValues = model.computeExpectedPosteriorCIFs(times=trials_times)
cifValuesKS = epmcifValues[trialForGOF][neuronForGOF]
spikesTimesKS = spikes_times[trialForGOF][neuronForGOF]
diffECDFsX, diffECDFsY, estECDFx, estECDFy, simECDFx, simECDFy, cb = gcnu_common.stats.pointProcesses.tests.KSTestTimeRescalingNumericalCorrection(spikesTimes=spikesTimesKS, cifTimes=trials_times, cifValues=cifValuesKS, gamma=ksTest_gamma)
title = "Trial {:d}, Neuron {:d}".format(trialForGOF, neuronForGOF)
fig = svGPFA.plot.plotUtilsPlotly.getPlotResKSTestTimeRescalingNumericalCorrection(diffECDFsX=diffECDFsX, diffECDFsY=diffECDFsY, estECDFx=estECDFx, estECDFy=estECDFy, simECDFx=simECDFx, simECDFy=simECDFy, cb=cb, title=title)
fig.show()

## 3.3 ROC predictive analysis

In [ ]:
dt = (trials_times[1] - trials_times[0]).item()
pk = cifValuesKS.detach().numpy() * dt
bins = pd.interval_range(start=trials_start_time, end=trials_end_time, periods=len(pk))
cutRes, _ = pd.cut(spikesTimesKS.tolist(), bins=bins, retbins=True)
Y = cutRes.value_counts().values
fpr, tpr, thresholds = sklearn.metrics.roc_curve(Y, pk, pos_label=1)
roc_auc = sklearn.metrics.auc(fpr, tpr)
title = "Trial {:d}, Neuron {:d}".format(trialForGOF, neuronForGOF)
fig = svGPFA.plot.plotUtilsPlotly.getPlotResROCAnalysis(fpr=fpr, tpr=tpr, auc=roc_auc, title=title)
fig.show()